In [1]:
import pandas as pd
import numpy as np
from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers import pipeline
from tqdm.auto import tqdm
from torch.utils.data import Dataset, DataLoader
import json
import jsonlines
import itertools


In [2]:
NER = ["EFFECT"]

In [3]:
l = []
for ner in NER:
	test_sentences = pd.read_json(f"../NER/{ner}/test.json", lines=True)
	test_output = pd.read_json(f"../NER_output/{ner}/test_outputs.json")
	merged = pd.merge(test_sentences, test_output, left_index=True, right_index=True)
	merged.loc[:,"ner"] = ner
	l.append(merged)

df = pd.concat(l)

In [4]:
df = df.reset_index().drop(columns=["index"])

In [5]:
df

,tokens,ner_tags,id,predictions,label_ids,word_ids,ner
0,"[Together, ,, these, results, indicate, that, ...","[O, O, O, O, O, O, O, O, O, O, O, O, B, I, O, ...",0,"[[-1.0871022939682, -0.6585953831672661, 1.041...","[-100, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, -100, 2, ...","[None, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 9, 10, 10...",EFFECT
1,"[It, should, be, noted, that, strains, FLEC03,...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...",1,"[[-2.040106773376465, -1.953450918197631, 3.35...","[-100, 2, 2, 2, 2, 2, 2, 2, -100, -100, 2, 2, ...","[None, 0, 1, 2, 3, 4, 5, 6, 6, 6, 7, 8, 8, 8, ...",EFFECT
2,"[90, ,, 18B6, ,, 66A, and, Hapalosiphon, hiber...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...",2,"[[-2.121238708496093, -1.8500920534133911, 3.1...","[-100, 2, 2, 2, -100, 2, 2, -100, 2, 2, -100, ...","[None, 0, 1, 2, 2, 3, 4, 4, 5, 6, 6, 6, 6, 6, ...",EFFECT
3,"[The, strain, N., spumigena, CCY9414, was, ini...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...",3,"[[-1.98106563091278, -1.995223522186279, 3.556...","[-100, 2, 2, 2, -100, 2, -100, -100, -100, 2, ...","[None, 0, 1, 2, 2, 3, 3, 3, 3, 4, 4, 4, 4, 5, ...",EFFECT
4,"[HPLC, analysis, showed, maximum, IAA, product...","[O, O, O, O, O, O, O, O, O, O]",4,"[[-1.137577176094055, -1.115126013755798, 2.06...","[-100, 2, 2, 2, 2, 2, 2, 2, 2, -100, -100, -10...","[None, 0, 1, 2, 3, 4, 5, 6, 7, 7, 7, 7, 8, 9, ...",EFFECT
...,...,...,...,...,...,...,...
722,"[The, experiments, took, place, in, sealed, 1-...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...",722,"[[-2.145196199417114, -1.8208420276641841, 3.4...","[-100, 2, 2, 2, 2, 2, 2, 2, -100, -100, 2, 2, ...","[None, 0, 1, 2, 3, 4, 5, 6, 6, 6, 7, 8, 9, 10,...",EFFECT
723,"[Pfs230, ,, a, surface, antigen, of, gametocyt...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...",723,"[[-1.3918730020523071, -1.341086864471435, 2.2...","[-100, 2, -100, -100, 2, 2, 2, 2, 2, 2, -100, ...","[None, 0, 0, 0, 1, 2, 3, 4, 5, 6, 6, 7, 8, 9, ...",EFFECT
724,"[Under, SSP126, ,, 77.3, %, (, 2.49, ×, 10^6, ...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...",724,"[[-1.55739426612854, -1.507947921752929, 2.931...","[-100, 2, 2, -100, 2, 2, -100, -100, 2, 2, 2, ...","[None, 0, 1, 1, 2, 3, 3, 3, 4, 5, 6, 6, 6, 7, ...",EFFECT
725,"[Accordingly, ,, E., coli, MG1655, was, a, bet...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O]",725,"[[-1.053021907806396, -1.6610616445541382, 2.2...","[-100, 2, 2, 2, -100, 2, 2, -100, -100, 2, 2, ...","[None, 0, 1, 2, 2, 3, 4, 4, 4, 5, 6, 7, 8, 8, ...",EFFECT


In [6]:
label_ids = df[["label_ids"]]

In [7]:
annotations = []
current_annotation = None

for j in range(len(df)):
	for i in zip(df.loc[j, "tokens"], df.loc[j, "ner_tags"]):
		if i[1] != "O":
			if current_annotation is None:
				current_annotation = { "text": i[0]}
			else:
				current_annotation["text"] += " " + i[0]
		else:
			if current_annotation is not None:
				annotations.append(current_annotation)
				current_annotation = None

if current_annotation is not None:
	annotations.append(current_annotation)


### Predict sentences again

In [8]:
ner = "EFFECT"

In [9]:
path= f"../NER_output/{ner}"

tokenizer = AutoTokenizer.from_pretrained(path, model_max_length=512)
model = AutoModelForTokenClassification.from_pretrained(path)

nlp = pipeline(task='ner', model=model, tokenizer=tokenizer,
               aggregation_strategy="average", device=0)


In [10]:
# Specify the path to the JSONL file
jsonl_file = f"../NER/{ner}/test.jsonls"

predictions = []
# Open the JSONL file
with jsonlines.open(jsonl_file) as reader:
    # Iterate over each line in the file
    for line in reader:
        for l in line:
            # Get the value from the "text" field
            text = l["data"]["text"]
            # Pass the value to the nlp() function
            print(text)
            predict = nlp(text)
            predictions.append(predict)
            print(predict)

Together, these results indicate that (i) B. distachyon increased phytosiderophore production in response to bacterial colonization and (ii) SBW25 consumed or otherwise degraded these phytosiderophores, resulting in overall lower abundances within the root exudate solution.
[{'entity_group': 'I', 'score': 0.5481588, 'word': 'colonization', 'start': 119, 'end': 131}, {'entity_group': 'I', 'score': 0.71386725, 'word': 'exudate', 'start': 257, 'end': 264}]
It should be noted that strains FLEC03, RI-406, RI-124, RI-193, RI-198, and MRS6 share a common source of isolation (meat), whereas the other strains were isolated from fermented nonmeat products (except for CRL705).
[]
90, 18B6, 66A and Hapalosiphon hibernicus BZ-3-1 were all more similar to one another than they were to other mcyB1 or mcyC sequences (Figure 7b).
[]
The strain N. spumigena CCY9414 was initially isolated from samples collected from surface water in the Bornholm Sea ( Hayes and Barker, 1997 ; Stal et al., 2003 ; Voss et 

/home/gomez/miniconda3/envs/torch/lib/python3.11/site-packages/transformers/pipelines/base.py:1090: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


[]
To determine a suitable time point for RNA extraction, growth curve analysis was performed on B. vietnamiensis strain G4 cultured in TSB (control) and TSB containing HEPB at 0.75× and 0.5× MIC (test condition) for 24 h as described above.
[]
SAM-B isolated from a mineral cold spring located in Tyva, southern Siberia
[]
Innate immunity is crucial for the elimination of SARS-CoV-2, the removal of dead cells, and restoring tissue function, as well as coordinating/sustaining the protective adaptive immune responses of virus-specific T and B lymphocytes [3,4].
[{'entity_group': 'I', 'score': 0.61461645, 'word': 'innate immunity', 'start': 0, 'end': 15}, {'entity_group': 'I', 'score': 0.6672671, 'word': 'elimination of', 'start': 35, 'end': 49}, {'entity_group': 'I', 'score': 0.67472935, 'word': 'removal of dead cells', 'start': 66, 'end': 87}, {'entity_group': 'I', 'score': 0.67726415, 'word': 'restoring tissue function', 'start': 93, 'end': 118}, {'entity_group': 'I', 'score': 0.6460652

In [11]:
predictions = [p for p in predictions if p]

In [12]:
predictions

[[{'entity_group': 'I',
   'score': 0.5481588,
   'word': 'colonization',
   'start': 119,
   'end': 131},
  {'entity_group': 'I',
   'score': 0.71386725,
   'word': 'exudate',
   'start': 257,
   'end': 264}],
 [{'entity_group': 'I',
   'score': 0.5169901,
   'word': 'proliferation',
   'start': 38,
   'end': 51}],
 [{'entity_group': 'I',
   'score': 0.55297965,
   'word': 'degradation',
   'start': 196,
   'end': 207}],
 [{'entity_group': 'I',
   'score': 0.61461645,
   'word': 'innate immunity',
   'start': 0,
   'end': 15},
  {'entity_group': 'I',
   'score': 0.6672671,
   'word': 'elimination of',
   'start': 35,
   'end': 49},
  {'entity_group': 'I',
   'score': 0.67472935,
   'word': 'removal of dead cells',
   'start': 66,
   'end': 87},
  {'entity_group': 'I',
   'score': 0.67726415,
   'word': 'restoring tissue function',
   'start': 93,
   'end': 118},
  {'entity_group': 'I',
   'score': 0.6460652,
   'word': 'sustaining the protective adaptive immune responses of virus - sp

In [13]:
for prediction in predictions:
    pred = prediction[0]["word"]
    count = 0
    pred = pred.replace(" - ","-").replace("- ","-").replace(" -","-")
    for annotation in annotations:
        ann = annotation["text"]
        ann = ann.lower()
        if ann in pred:
            count +=1
    if count ==0:
        print(pred)

colonization
enhancement
metal
plant
stress
aggregation
availability
storage rot
production
plant
promote plant
import of
moisture
biofilm
biofilm
activity
health
physiological processes
secretion
reversal
catabolism
production
binding
contagion
transfer
hypersensitive
anti
ec-coupling
colonization
cr ( vi
stress response
salts
immune responses
degrading
release of tnf
priming
antipathogenic
biofilm
isr triggered
binding
oxidation of
inhibition
immune response
periplasmic ph
biology
adoption
resistance
promoting
acidification
positive
signaling
formation
term
survive
survival in
nodule number
signaling
resistance to
plant
neutralization antibodies
production
reduction
pollen
neuroprotective
retention
concentrations
salinity
cereal seed-borne diseases
transfer
soybean cyst
division
biomass
biomass
drop
salt
transfer
disease indexes
productivity
resistance to the heat stress of
lactam
seedlings
occurrence of
uptake
oligomers
binding
ciliogenesis
acid
biological
biofilm
host
neurodegenera

In [14]:
merged

,tokens,ner_tags,id,predictions,label_ids,word_ids,ner
0,"[Together, ,, these, results, indicate, that, ...","[O, O, O, O, O, O, O, O, O, O, O, O, B, I, O, ...",0,"[[-1.0871022939682, -0.6585953831672661, 1.041...","[-100, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, -100, 2, ...","[None, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 9, 10, 10...",EFFECT
1,"[It, should, be, noted, that, strains, FLEC03,...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...",1,"[[-2.040106773376465, -1.953450918197631, 3.35...","[-100, 2, 2, 2, 2, 2, 2, 2, -100, -100, 2, 2, ...","[None, 0, 1, 2, 3, 4, 5, 6, 6, 6, 7, 8, 8, 8, ...",EFFECT
2,"[90, ,, 18B6, ,, 66A, and, Hapalosiphon, hiber...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...",2,"[[-2.121238708496093, -1.8500920534133911, 3.1...","[-100, 2, 2, 2, -100, 2, 2, -100, 2, 2, -100, ...","[None, 0, 1, 2, 2, 3, 4, 4, 5, 6, 6, 6, 6, 6, ...",EFFECT
3,"[The, strain, N., spumigena, CCY9414, was, ini...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...",3,"[[-1.98106563091278, -1.995223522186279, 3.556...","[-100, 2, 2, 2, -100, 2, -100, -100, -100, 2, ...","[None, 0, 1, 2, 2, 3, 3, 3, 3, 4, 4, 4, 4, 5, ...",EFFECT
4,"[HPLC, analysis, showed, maximum, IAA, product...","[O, O, O, O, O, O, O, O, O, O]",4,"[[-1.137577176094055, -1.115126013755798, 2.06...","[-100, 2, 2, 2, 2, 2, 2, 2, 2, -100, -100, -10...","[None, 0, 1, 2, 3, 4, 5, 6, 7, 7, 7, 7, 8, 9, ...",EFFECT
...,...,...,...,...,...,...,...
722,"[The, experiments, took, place, in, sealed, 1-...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...",722,"[[-2.145196199417114, -1.8208420276641841, 3.4...","[-100, 2, 2, 2, 2, 2, 2, 2, -100, -100, 2, 2, ...","[None, 0, 1, 2, 3, 4, 5, 6, 6, 6, 7, 8, 9, 10,...",EFFECT
723,"[Pfs230, ,, a, surface, antigen, of, gametocyt...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...",723,"[[-1.3918730020523071, -1.341086864471435, 2.2...","[-100, 2, -100, -100, 2, 2, 2, 2, 2, 2, -100, ...","[None, 0, 0, 0, 1, 2, 3, 4, 5, 6, 6, 7, 8, 9, ...",EFFECT
724,"[Under, SSP126, ,, 77.3, %, (, 2.49, ×, 10^6, ...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...",724,"[[-1.55739426612854, -1.507947921752929, 2.931...","[-100, 2, 2, -100, 2, 2, -100, -100, 2, 2, 2, ...","[None, 0, 1, 1, 2, 3, 3, 3, 4, 5, 6, 6, 6, 7, ...",EFFECT
725,"[Accordingly, ,, E., coli, MG1655, was, a, bet...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O]",725,"[[-1.053021907806396, -1.6610616445541382, 2.2...","[-100, 2, 2, 2, -100, 2, 2, -100, -100, 2, 2, ...","[None, 0, 1, 2, 2, 3, 4, 4, 4, 5, 6, 7, 8, 8, ...",EFFECT


In [15]:
merged

,tokens,ner_tags,id,predictions,label_ids,word_ids,ner
0,"[Together, ,, these, results, indicate, that, ...","[O, O, O, O, O, O, O, O, O, O, O, O, B, I, O, ...",0,"[[-1.0871022939682, -0.6585953831672661, 1.041...","[-100, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, -100, 2, ...","[None, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 9, 10, 10...",EFFECT
1,"[It, should, be, noted, that, strains, FLEC03,...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...",1,"[[-2.040106773376465, -1.953450918197631, 3.35...","[-100, 2, 2, 2, 2, 2, 2, 2, -100, -100, 2, 2, ...","[None, 0, 1, 2, 3, 4, 5, 6, 6, 6, 7, 8, 8, 8, ...",EFFECT
2,"[90, ,, 18B6, ,, 66A, and, Hapalosiphon, hiber...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...",2,"[[-2.121238708496093, -1.8500920534133911, 3.1...","[-100, 2, 2, 2, -100, 2, 2, -100, 2, 2, -100, ...","[None, 0, 1, 2, 2, 3, 4, 4, 5, 6, 6, 6, 6, 6, ...",EFFECT
3,"[The, strain, N., spumigena, CCY9414, was, ini...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...",3,"[[-1.98106563091278, -1.995223522186279, 3.556...","[-100, 2, 2, 2, -100, 2, -100, -100, -100, 2, ...","[None, 0, 1, 2, 2, 3, 3, 3, 3, 4, 4, 4, 4, 5, ...",EFFECT
4,"[HPLC, analysis, showed, maximum, IAA, product...","[O, O, O, O, O, O, O, O, O, O]",4,"[[-1.137577176094055, -1.115126013755798, 2.06...","[-100, 2, 2, 2, 2, 2, 2, 2, 2, -100, -100, -10...","[None, 0, 1, 2, 3, 4, 5, 6, 7, 7, 7, 7, 8, 9, ...",EFFECT
...,...,...,...,...,...,...,...
722,"[The, experiments, took, place, in, sealed, 1-...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...",722,"[[-2.145196199417114, -1.8208420276641841, 3.4...","[-100, 2, 2, 2, 2, 2, 2, 2, -100, -100, 2, 2, ...","[None, 0, 1, 2, 3, 4, 5, 6, 6, 6, 7, 8, 9, 10,...",EFFECT
723,"[Pfs230, ,, a, surface, antigen, of, gametocyt...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...",723,"[[-1.3918730020523071, -1.341086864471435, 2.2...","[-100, 2, -100, -100, 2, 2, 2, 2, 2, 2, -100, ...","[None, 0, 0, 0, 1, 2, 3, 4, 5, 6, 6, 7, 8, 9, ...",EFFECT
724,"[Under, SSP126, ,, 77.3, %, (, 2.49, ×, 10^6, ...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...",724,"[[-1.55739426612854, -1.507947921752929, 2.931...","[-100, 2, 2, -100, 2, 2, -100, -100, 2, 2, 2, ...","[None, 0, 1, 1, 2, 3, 3, 3, 4, 5, 6, 6, 6, 7, ...",EFFECT
725,"[Accordingly, ,, E., coli, MG1655, was, a, bet...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O]",725,"[[-1.053021907806396, -1.6610616445541382, 2.2...","[-100, 2, 2, 2, -100, 2, 2, -100, -100, 2, 2, ...","[None, 0, 1, 2, 2, 3, 4, 4, 4, 5, 6, 7, 8, 8, ...",EFFECT


In [16]:
## TODO: also split the annotations by "-"
# Create an empty dataframe to store the results
results_df = pd.DataFrame(columns=['NER', 'Prediction'])

ners = ["COMPOUND", "DISEASE", "EFFECT", "ISOLATE", "MEDIUM", "METABOLITE", "ORGANISM",
        "PHENOTYPE", "PROTEIN", "SPECIES", "STRAIN"]

for ner in ners:
    test_sentences = pd.read_json(f"../NER/{ner}/test.json", lines=True)
    test_output = pd.read_json(f"../NER_output/{ner}/test_outputs.json")
    merged = pd.merge(test_sentences, test_output, left_index=True, right_index=True)
    merged.loc[:, "ner"] = ner

    annotations = []
    current_annotation = None

    for j in range(len(merged)):
        for i in zip(merged.loc[j, "tokens"], merged.loc[j, "ner_tags"]):
            if i[1] != "O":
                if current_annotation is None:
                    current_annotation = {"text": i[0]}
                else:
                    current_annotation["text"] += " " + i[0]
            else:
                if current_annotation is not None:
                    annotations.append(current_annotation)
                    current_annotation = None

    if current_annotation is not None:
        annotations.append(current_annotation)

    path = f"../NER_output/{ner}"

    tokenizer = AutoTokenizer.from_pretrained(path, model_max_length=512)
    model = AutoModelForTokenClassification.from_pretrained(path)

    nlp = pipeline(task='ner', model=model, tokenizer=tokenizer,
                   aggregation_strategy="average", device=0)

    # Specify the path to the JSONL file
    jsonl_file = f"../NER/{ner}/test.jsonls"

    predictions = []
    # Open the JSONL file
    with jsonlines.open(jsonl_file) as reader:
        # Iterate over each line in the file
        for line in reader:
            for l in line:

                # Get the value from the "text" field
                text = l["data"]["text"]
                # Pass the value to the nlp() function
                predict = nlp(text)
                predictions.append(predict)
    predictions = [p for p in predictions if p]

    # Store the predictions in the dataframe
    for prediction in predictions:
        pred = prediction[0]["word"]
        count = 0
        pred = pred.replace(" - ", "-").replace("- ", "-").replace(" -", "-").lower()
        for annotation in annotations:
            ann = annotation["text"]
            ann = ann.lower()
            if ann in pred:
                count += 1
        if count == 0:
            results_df = pd.concat([results_df, pd.DataFrame({'NER': [ner], 'Prediction': [pred]})], ignore_index=True)

    annotations_set = [i["text"].lower().split(" ") for i in annotations]
    annotations_set = set(itertools.chain.from_iterable(annotations_set))

    predictions_set = [i[0]["word"].lower().replace(" - ", "-").replace("- ", "-").replace(" -", "-").split(" ") for i in predictions]
    predictions_set = set(itertools.chain.from_iterable(predictions_set))

    ann_not_pred = annotations_set.difference(predictions_set)
    pred_not_ann = predictions_set.difference(annotations_set)

    with open(f"ner_annotation/{ner}_ann_not_pred.txt", "w") as f:
        for i in ann_not_pred:
            f.write(i + "\n")
    with open(f"ner_annotation/{ner}_pred_not_ann.txt", "w") as f:
        for i in pred_not_ann:
            f.write(i + "\n")

/home/gomez/miniconda3/envs/torch/lib/python3.11/site-packages/transformers/pipelines/base.py:1090: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
/home/gomez/miniconda3/envs/torch/lib/python3.11/site-packages/transformers/pipelines/base.py:1090: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
/home/gomez/miniconda3/envs/torch/lib/python3.11/site-packages/transformers/pipelines/base.py:1090: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
/home/gomez/miniconda3/envs/torch/lib/python3.11/site-packages/transformers/pipelines/base.py:1090: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
/home/gomez/miniconda3/envs/torch/lib/python